# PART I

## DATA PROCESSING

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.style.use('ggplot')
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16
plt.rcParams["legend.edgecolor"] = 'black'
plt.rcParams["legend.fontsize"] = 13

train_data = pd.read_csv("IA1_train.csv")
val_data = pd.read_csv("IA1_dev.csv")

In [ ]:
def normalize(data, column, mu_dict, sigma_dict):
    return (data[column] - mu_dict[column])/sigma_dict[column]

        
def dataProcessing(data, mu_dict, sigma_dict):
    data.insert(loc=0, column="bias", value=1)
        
    # Drop unused features
    data = data.drop(columns=["id", "date", "yr_renovated"])
    
    for column in data: 
        if column not in ['bias', 'waterfront', 'price']:
            data[column] = normalize(data, column, mu_dict, sigma_dict)
    
    return data


def dateSplit(data):
    data['month'] = [int(x.split("/")[0]) for x in data['date'].values] # month
    data['day'] = [int(x.split("/")[1]) for x in data['date'].values] # day
    data['year'] = [int(x.split("/")[2]) for x in data['date'].values] # year
    return data


def reno(data):
    data['age_since_renovated'] = data.apply(lambda d: ((d["year"] - d["yr_built"]) if (d["yr_renovated"] == 0) 
                                                        else (d["year"] - d["yr_renovated"])), axis=1)
    return data


# Split date
train_data = dateSplit(train_data)
val_data = dateSplit(val_data)

# Age since renovated
train_data = reno(train_data)
val_data = reno(val_data)

# Find mu and sigma for train -> put into dict
mu_dict = {}
sigma_dict = {}

for col in train_data.columns.values:
    if col not in ['id', 'date']:
        mu_dict[col] = train_data[col].mean()
        sigma_dict[col] = train_data[col].std()

        
# Finish process data
train_norm = dataProcessing(train_data, mu_dict, sigma_dict)
val_norm = dataProcessing(val_data, mu_dict, sigma_dict)

In [ ]:
train_norm.head()

In [ ]:
val_norm.head()

## DATA EXPLORATION

### Feature Correlation (Report Only)

In [ ]:
import seaborn as sns
# cant add to script since it uses seaborn

corr_matrix = train_norm.corr()
plt.figure(figsize=(14,12))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='BuPu')
plt.show()

# MODEL TRAIN

## 1a

In [ ]:
features = train_norm.columns.drop("price")
n_features = len(features)

X_norm = train_norm[features]
y = train_norm["price"].to_frame()

X_val = val_norm[features]
y_val = val_norm["price"].to_frame()

# Configuration
learning_rates = [1, 0.178, 0.15, 0.1, 0.05, 0.01, 0.001, 0.0001]
n_iterations = 4000
n = len(X_norm)
epsilonLow = 10**(-2) # converge values
epsilonHigh = 10**5 # diverge value

w_init = np.zeros(n_features).reshape(n_features,1)
w = w_init

mse_lr = {}
w_lr = {}
diverge = []

for lr in learning_rates: # iterate over learning rates
    w = w_init
    mse_train = []
    div = False
    print("Learning Rate: ", lr) # DELETE
    for iter in range(n_iterations):
        # Rename the product to match y label
        y_hat = (X_norm.dot(w)).rename(columns={0: "price"})

        grads = 2/n * X_norm.T.dot(y_hat - y)
        w = w - lr * grads

        y_pred = (X_norm.dot(w)).rename(columns={0: "prices"})
        mse = float(((y - y_pred) ** 2).sum() / n)
        mse_train.append(mse)

        magLw = np.linalg.norm(grads.values) # magnitude of gradient (change in weights)

        # Early stop due to tiny change or diverging
        if magLw < epsilonLow:
            print("Converge, iter: ", iter) # DELETE
            break

        if magLw > epsilonHigh:
            print("Diverge, iter: ", iter) # DELETE
            div = True
            break
    
    diverge.append(div)
    mse_lr[str(lr)] = mse_train
    w_lr[str(lr)] = w

In [ ]:
plt.figure(figsize=(14,8))
for i, (lr, mse) in enumerate(mse_lr.items()):
    if not diverge[i]: # dont plot diverging val
        plt.plot(np.arange(1,len(mse)+1), mse, label=lr, linewidth=3)
    
plt.legend(title="LR")   
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.title("MSE per Iteration by Learning Rates (Figure 1)")

### Report Question:
1a) Which learning rate or learning rates did you observe to be good for this particular dataset? What learning rates (if any) make gradient descent diverge?

It seems that learning rates approaching $0**{+}$ from 0.178 all were on the path of convergence (but not necessarily reached convergence). A learning rate of 0.178 resulted in the best minimization of the MSE (discussed in part 1b) in 894 iterations. But as values grew smaller, the model continued to converge with a larger MSE and more iterations. This is correct since our steps are becoming smaller and thus learning becomes much slow (would need more iterations to find it gradient value for converge). All learning rates greater than or equal 0.179 caused divergence. A learning rate of 0.179 diverged in 1515 iterations, while a learning rate of 0.178 converged in 894 iterations. As the learning rate increase, the number of iterations needed to diverge becomes smaller (i.e. a learning rate of 0.3 diverged in 11 iterations, 0.5 in 7 iterations, and 1.0 in 4 iterations). The model seemed to have the best convergence (0.178) very close to where it also began to diverge (0.179).

## 1b

In [ ]:
cur_min = 1000000
best_model_norm = " "
print("Figure 2:\n")
for i, (lr, w) in enumerate(w_lr.items()):
    if not diverge[i]:
        y_pred = (X_val.dot(w)).rename(columns={0: "prices"})
        mse = float(((y_val - y_pred) ** 2).sum() / len(y_val))
        print("Learning Rate:", lr + ", MSE:", mse)
        if mse < cur_min:
            cur_min = mse
            best_model_norm = lr

### Report Question 
1b) Which learning rate leads to the best validation MSE? Between different convergent learning rates, how should we choose one if the validation MSE is nearly identical?

Using a learning rate of 0.178 resulted in the lowest MSE (a value of approximately 4.5144). If two different convergent learning rates have nearly the same MSE for the validation data, we should choose the larger learning step. This is because the two learning rates will give the model similar perform, but the larger learning rate will result in a faster training time. For our model, learning rates 0.15 and 0.05 had a difference of 0.0001 in their MSE. However, the model utilizing a learning rate of 0.15 converged early in 882 iterations while the model utlizing 0.05 converged early in 2646 iterations. The extra iterations for the second model ($lr$ = $0.05$) would be unecessary since the MSE difference between them is minimal and not worth the cost trade off of unnecessary training for an extended period of time.


## 1c

In [ ]:
# Drop w0, print all weights for best model\
print("Best model has a learning rate of: ", best_model_norm, ", with feature importance below:\n")
feat_importance = w_lr[best_model_norm].rename(columns={"price":"weights"}).drop(index="bias")
for i, idx in enumerate(feat_importance.index.values):
    print(idx + ":", feat_importance.iloc[i].values[0])

### Report Question
1c) What features are the most important in deciding the house prices according to the learned weights?

For our model, the most important feature is `waterfront` (indicating whether the apartment was overlooking the waterfront or not) with a weight value of approximately 3.3595. Following this, the `grade` (a scale of building construction design and quality) with a weight value of approximately 1.1139. The third most import feature is `yr_built` (initial house build year) with a weight value of approximately -0.88245. The first two features both increase the house price, while the third decreases the house price.

## Using scikit-learn

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(X_norm, y)

y_pred = lin_reg.predict(X_val)

print(y_pred)
len(y_pred)

print("MSE: %.2f" % mean_squared_error(y_val, y_pred))
print("w: " + str(lin_reg.coef_))


# PART II

## A. Training with non-normalized data

### DATA PREPROCESSING

In [ ]:
features = train_data.columns.drop(["price", "id", "date", "yr_renovated"])
n_features = len(features)

X_non_norm = train_data[features]
y = train_data["price"].to_frame()

X_val_non_norm = val_data[features]
y_val = val_data["price"].to_frame()

In [ ]:
X_val_non_norm

### MODEL TRAIN

### a)
Manual experiments show that the learning rate that yields that best model 
lies somewhere in the range [10**(-11),10**(-10)). Using binary search to find 
the best learning rate.

In [ ]:
def fit(X, y, n_iters: int, lr: float, epsilon_low: float, epsilon_high: float):
    converge = True
    n = X.shape[0]
    n_features = X.shape[1]
    w = np.zeros(n_features).reshape(n_features,1)
    
    mse_train = []
    
    for iter in range(n_iters):
        # Rename the product to match y label
        y_hat = (X.dot(w)).rename(columns={0: "price"})
        
        grads = 2/n * X.T.dot(y_hat - y)
        w = w - lr * grads
        
        y_pred = (X.dot(w)).rename(columns={0: "prices"})
        mse = float(((y - y_pred) ** 2).sum() / n)
        mse_train.append(mse)
        
        magLw = np.linalg.norm(grads.values)
        
        # Early stop due to tiny change or diverging
        if magLw < epsilon_low:
            print("Learning rate %s plateau at iter #%d, magLW = %f" % (str(lr), iter, magLw))
            break
        
        if magLw > epsilon_high:
            print("Learning rate %s Diverge at iter #%d, magLw = %f" % (str(lr), iter, magLw))
            converge = False
            break
        
    return converge, mse_train, w

# 
for lr in [10**(-1),10**(-2),10**(-3), 10**(-4),10**(-5),10**(-6),10**(-7),10**(-8),10**(-9),10**(-10)]:
    r_converge, r_mse_train, r_w = fit(X_non_norm, y, n_iters=4000,
                                                lr=lr,
                                                epsilon_low=10**(-6), epsilon_high=10**10)


# Use binary search to search for optimal learning rate
mse_lr = {}
w_lr = {}
diverge = []

max_depth = 10
l = 10**(-11)
r = 10**(-10)
cur_depth = 0
l_converge, l_mse_train, l_w = fit(X_non_norm, y, n_iters=4000, 
                                            lr=l, 
                                            epsilon_low=10**(-6), epsilon_high=10**10)

diverge.append(not l_converge)
mse_lr[str(l)] = l_mse_train
w_lr[str(l)] = l_w

r_converge, r_mse_train, r_w = fit(X_non_norm, y, n_iters=4000, 
                                            lr=r, 
                                            epsilon_low=10**(-6), epsilon_high=10**10)

diverge.append(not r_converge)
mse_lr[str(r)] = r_mse_train
w_lr[str(r)] = r_w

while cur_depth < max_depth:
    m = (l + r) / 2
    
    converge, mse_train, w = fit(X_non_norm, y, n_iters=4000, lr=m, epsilon_low=10**(-6), epsilon_high=10**10)
    diverge.append(not converge)
    mse_lr[str(m)] = mse_train
    w_lr[str(m)] = w
    
    if l_mse_train[-1] < r_mse_train[-1]:
        r = m
        r_converge = converge
        r_mse_train = mse_train
        r_w = w
    else:
        l = m
        l_converge = converge
        l_mse_train = mse_train
        l_w = w
    
    cur_depth += 1


In [ ]:
mse_lr

In [ ]:
plt.figure(figsize=(14,8))
for i, (lr, mse) in enumerate(mse_lr.items()):
    if not diverge[i]: # dont plot diverging val
        plt.plot(np.arange(1,len(mse)+1), mse, label=lr, linewidth=3)
    
plt.legend(title="LR")   
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.title("MSE per Iteration by Learning Rates")

### b) The best learning rate and its resulting MSE on the training and validation data

In [ ]:
# On training data
cur_min = 1000000
best_model_non_norm = " "

for i, (lr, w) in enumerate(w_lr.items()):
    if not diverge[i]:
        y_pred = (X_non_norm.dot(w)).rename(columns={0: "prices"})
        mse = float(((y - y_pred) ** 2).sum() / len(y))
        print("Learning Rate:", lr + ", MSE:", mse)
        if mse < cur_min:
            cur_min = mse
            best_model_non_norm = lr

In [ ]:
# On validation data
cur_min = 1000000
best_model_non_norm = " "

for i, (lr, w) in enumerate(w_lr.items()):
    if not diverge[i]:
        y_pred = (X_val_non_norm.dot(w)).rename(columns={0: "prices"})
        mse = float(((y_val - y_pred) ** 2).sum() / len(y_val))
        print("Learning Rate:", lr + ", MSE:", mse)
        if mse < cur_min:
            cur_min = mse
            best_model_non_norm = lr

In [ ]:
# Drop w0, print all weights for best model
print("Best model has a learning rate of: ", best_model_non_norm, ", with feature importance below:\n")
feat_importance = w_lr[best_model_non_norm].rename(columns={"price":"weights"}).drop(index="bias")
for i, idx in enumerate(feat_importance.index.values):
    print(idx + ":", feat_importance.iloc[i].values[0])

## B. Redundancy in features

**Expectation**:
- The importance of sqft_living when using only one feature will be higher than 
that when both features are used. During the learning, think of the grad with 
respect to sqft_living15 as a vector whose projection on the grad with respect 
to sqft_living is non-zero (because they are correlated), we can decompose it 
to the projection plus another vector. When we use two features, the importance 
of the "underlying feature" (the correlated part of them) is split and carried by two 
weights. Therefore, when only one feature is used, its weight will carry the 
entire importance of the "underlying feature".
- This phenomenon can result in inaccurate interpretation of feature importance 
if we are not aware of the correlation. (need more reasoning here!)

**Questions**: 
1. Correlation btw the two features is 0.76, which is not really "high". What 
should be the appropriate correlation threshold?
2. In industrial settings, should we make sense of the correlations between 
features or just blindly rely on the stats (hence, machine learning, not human 
learning). If the reasoning is necessary, what is the justification for this 
particular correlation?

In [ ]:
# Drop redundant feature
redundant_features = ["sqft_living15"]
X_norm_wo_r_train = X_norm.drop(redundant_features, axis=1)
X_norm_wo_r_val = X_val.drop(redundant_features, axis=1)

# Train
converge, mse_train, w = fit(X_norm_wo_r_train, y, n_iters=4000, 
                             lr=0.178, 
                             epsilon_low=10**(-6), epsilon_high=10**10)


In [ ]:
y_norm_wo_r_pred = (X_norm_wo_r_val.dot(w)).rename(columns={0: "prices"})
mse = float(((y_val - y_norm_wo_r_pred) ** 2).sum() / len(y_val))

print(mse)
print(w)
print(converge)

The importance of "sqft_living" when using alone is indeed higher than that when 
using two features. In particular, w_{sqft_living | non-redundant} = 0.8 and 
w_{sqft_living | redundant} = 0.77 and w_{sqft_living15 | non-redundant} = 0.14. 

